In [ ]:
# 导入各层smti与气候因子
import pandas as pd
import xgboost as xgb
import shap
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
# 中文显示问题
plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 1. 读取数据
e = pd.read_excel('D:/Permafrost/temp/var/e/e.xlsx', index_col=0)
t2m = pd.read_excel('D:/Permafrost/temp/var/t2m/t2m.xlsx', index_col=0)
tp = pd.read_excel('D:/Permafrost/temp/var/tp/tp.xlsx', index_col=0)
sp = pd.read_excel('D:/Permafrost/temp/var/sp/sp.xlsx', index_col=0)
sde = pd.read_excel('D:/Permafrost/temp/var/sde/sde.xlsx', index_col=0)
v10 = pd.read_excel('D:/Permafrost/temp/var/v10/v10.xlsx', index_col=0)
str = pd.read_excel('D:/Permafrost/temp/var/str/str.xlsx', index_col=0)
strd = pd.read_excel('D:/Permafrost/temp/var/strd/strd.xlsx', index_col=0)
ssr = pd.read_excel('D:/Permafrost/temp/var/ssr/ssr.xlsx', index_col=0)
ssrd = pd.read_excel('D:/Permafrost/temp/var/ssrd/ssrd.xlsx', index_col=0)
ro = pd.read_excel('D:/Permafrost/temp/var/ro/ro.xlsx', index_col=0)
rh = pd.read_excel("D:/Permafrost/temp/var/relative_humidity.xlsx", index_col=0)
#
smti = pd.read_excel('D:/Permafrost/temp/stmi/stmi_not.xlsx', index_col=0)  # 读取SMTI数据


In [ ]:
# 绘制SHAP值图
import shap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Times New Roman']  # 指定默认
plt.rcParams['axes.unicode_minus'] = False
def plt_shap(shap_values, X_test, r2, rmse, mae):
    # 计算SHAP值相关数据
    mean_shap = np.mean(np.abs(shap_values.values), axis=0)
    feature_names = ['e', 't2m', 'tp', 'sp', 'sde', 'v10', 'str', 'strd', 'ssr', 'ssrd', 'ro', 'rh']
    shap_df = pd.DataFrame({'feature': feature_names, 'mean_shap': mean_shap}).sort_values('mean_shap', ascending=False)
    #
    sorted_features = shap_df['feature'].values
    shap_values_sorted = shap_values.values[:, [list(feature_names).index(f) for f in sorted_features]]
    mean_shap_sorted = shap_df['mean_shap'].values
    n_feat = len(sorted_features)
    mean_shap_percent = mean_shap_sorted / mean_shap_sorted.sum() * 100
    #
    mean_shap_sorted = shap_df['mean_shap'].values
    n_feat = len(sorted_features)
    mean_shap_percent = mean_shap_sorted / mean_shap_sorted.sum() * 100
    #
    # 创建画布和坐标系
    fig = plt.figure(figsize=(8, 8), dpi=600)  # 增加高度以容纳两个x轴
    ax = fig.add_axes([0.1, 0.15, 0.85, 0.75])  # 主坐标系

    # 创建共享y轴的第二个x轴（用于条形图）
    ax_bar = ax.twiny()  # 创建共享y轴的新坐标系（顶部x轴）
    # 设置图层顺序
    ax_bar.set_zorder(0)
    ax.set_zorder(1)
    ax.patch.set_alpha(0)  # 使主坐标系背景透明
    # 计算位置
    y_pos = np.arange(n_feat)[::-1]
    # 在顶部坐标系绘制条形图
    xlim_bar = mean_shap_sorted.max() * 1.05
    ax_bar.barh(y=y_pos, width=mean_shap_sorted, height=0.6, color='gray', alpha=0.3, edgecolor='none', zorder=0)
    # 设置条形图坐标轴
    ax_bar.set_xlim(0, xlim_bar)
    ax_bar.set_xlabel('Mean (|SHAP| value)', fontsize=30, labelpad=30)  # labelpad ：增加坐标轴标签与轴之间距离
    ax_bar.tick_params(axis='x', which='major',pad=10,labelsize=30)    # pad 增加刻度标签与轴的距离 label_size 坐标轴刻度标签字体大小
    # 在主坐标系绘制蜂群图 
    plt.sca(ax)  # 设置当前坐标系
    shap.summary_plot(shap_values.values, X_test, 
                      show=False, plot_size=None, cmap='RdYlBu_r')
    # 调节蜂群图 右侧颜色轴 标签字体大小 标签刻度字体大小
    fig = plt.gcf()
    ax = plt.gca()
    for child in fig.get_children():
        if isinstance(child, plt.Axes) and child.get_ylabel() == 'Feature value':
            # 设置颜色条标签字体大小
            child.set_ylabel('Feature value', fontsize=30)  # 修改标题字体大小
            # 设置刻度标签字体大小
            child.tick_params(axis='y', labelsize=30)  # 修改刻度字体大小
            break

    # 设置蜂群图坐标轴
    max_abs_shap = np.abs(shap_values_sorted).max()
    ax.set_xlim(-max_abs_shap * 1.5, max_abs_shap * 1.5)              # 调节 蜂群图 x 轴范围的
    ax.set_xlabel('SHAP value (impact on model output)', fontsize=30) # 设置蜂群图 x轴 标签
    ax.tick_params(axis='x', labelsize=30)                            #  设置蜂群图 x轴刻度标签的大小
    ax.set_yticks(y_pos)
    # 设置蜂群图 y轴 标签
    ax.set_yticklabels(sorted_features, fontsize=30) 
    # 设置蜂群图 y轴刻度标签大小
    ax.tick_params(axis='y', length=4,labelsize=30)
    # 添加条形图数值标签
    for i, (v, p) in enumerate(zip(mean_shap_sorted, mean_shap_percent)):
        ax_bar.text(x=0.01 * xlim_bar, y=y_pos[i]-0.01,
                s=f"{p:.1f}%",  #f"{v:.3f} ({p:.1f}%)"
                va='center', ha='left',
                fontsize=24, color='black',
                bbox=dict(facecolor='white', alpha=0, edgecolor='none', boxstyle='round,pad=0.2'))
    # 调节横向条形统计图 x轴 距离顶部的距离
    ax_bar.spines['top'].set_position(('axes', 1))
    ax.xaxis.labelpad = 15          # 增加蜂群图 x轴标签 与 轴之间距离
    plt.text(0.6, 0.35, f'R² = {r2:.4f}\nMae = {mae:.4f}\nRMSE = {rmse:.4f}', ha='left', va='top', transform=plt.gca().transAxes, fontsize=20, color='black')
    plt.show()

In [ ]:
# SHAP 依赖图
import math
import builtins
def plot_shap_dependence_grid(shap_values, X, layer_name,
                              ncols=4, figsize_scale=3.0,
                              color_by=None, point_size=6, alpha=0.6):
    features = list(X.columns)
    n_features = len(features)
    nrows = math.ceil(n_features / ncols)

    fig, axes = plt.subplots(nrows, ncols,
                             figsize=(ncols * figsize_scale, nrows * figsize_scale),
                             squeeze=False, dpi=600)

    # 预先把 X 的 ndarray 拿出来加速
    X_mat = X.values
    shap_mat = shap_values.values  # shape: (n_samples, n_features)
    color_feature_map = {}
    for idx, feat in enumerate(features):
        r, c = divmod(idx, ncols)
        ax = axes[r][c]

        j = X.columns.get_loc(feat)
        xj = X_mat[:, j]
        yj = shap_mat[:, j]

        # 选择着色变量
        color_arr = None
        color_feat_name = None
        if isinstance(color_by, builtins.str) and color_by != 'auto':
            if color_by in X.columns:
                color_arr = X[color_by].values
                color_feat_name = color_by
        elif color_by == 'auto':
            # 找与当前 SHAP_j 相关性|corr|最大的特征做着色（可近似交互强度）
            with np.errstate(invalid='ignore'):
                cors = [np.corrcoef(yj, X_mat[:, k])[0, 1] for k in range(X_mat.shape[1])]
            # 忽略自己
            cors[j] = np.nan
            k = int(np.nanargmax(np.abs(cors)))
            if np.isfinite(cors[k]):
                color_arr = X_mat[:, k]
                color_feat_name = X.columns[k]
        sc = ax.scatter(xj, yj, c=color_arr, s=point_size, alpha=alpha)
        ax.axhline(0, lw=0.8, ls='--', color='k')
        # ax.set_title(feat, fontsize=10)
        ax.set_xlabel(feat, fontsize=20)
        ax.set_ylabel('SHAP value', fontsize=20)
        ax.tick_params(axis='both', labelsize=20)

        if color_arr is not None:
            cb = plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.1)
            if color_feat_name is not None:
                cb.set_label(f'Color: {color_feat_name}', fontsize=8)
            cb.ax.tick_params(labelsize=8)
        color_feature_map[feat] = color_feat_name
    # 把空子图关掉
    for ax in axes.ravel()[n_features:]:
        ax.set_visible(False)
    plt.tight_layout(rect=[0, 0, 1, 0.98])
    return fig

In [ ]:
# SHAP 交互值图
import shap
import numpy as np
import pandas as pd
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.pyplot as plt
from math import ceil
def plot_shap_interactions(
    model, 
    X, 
    layer_name=None
):
    sample_size=500
    top_k=10
    top_n_to_plot=4
    ncols=2
    figsize=(10, 10)
    random_state=42
    print(f"\n>>> 绘制 SHAP 交互值: {layer_name if layer_name else ''}")
    # ---- 1. 初始化 TreeExplainer ----
    explainer = shap.TreeExplainer(model)
    # ---- 2. 抽样计算交互值 ----
    X_sample = shap.utils.sample(X, sample_size, random_state=random_state)
    shap_inter = explainer.shap_interaction_values(X_sample)
    # ---- 3. 计算交互强度矩阵 ----
    mean_abs_inter = np.mean(np.abs(shap_inter), axis=0)
    np.fill_diagonal(mean_abs_inter, 0.0)
    feature_names = X.columns.tolist()
    # ---- 5. 输出前 top_k 交互对 ----
    tri_inds = np.triu_indices_from(mean_abs_inter, k=1)
    pairs = list(zip(tri_inds[0], tri_inds[1]))
    scores = mean_abs_inter[tri_inds]
    top_idx = np.argsort(scores)[::-1][:top_k]

    # print(f"\nTop-{top_k} 交互对（{layer_name if layer_name else ''}）:")
    # top_pairs = []
    # for rank, idx in enumerate(top_idx, 1):
    #     i, j = pairs[idx]
    #     pair = (feature_names[i], feature_names[j], scores[idx])
    #     print(f"{rank:>2}. {pair[0]} × {pair[1]}  ->  {pair[2]:.6f}")
    #     top_pairs.append(pair)

    # ---- 6. 绘制最强交互散点图 ----
    best_i, best_j = pairs[top_idx[0]]
    sv_sample = shap.Explainer(model)(X_sample)
    # ---- 6. 把前4个交互对画到一张图里 ----
    m = min(top_n_to_plot, len(top_idx))
    nrows = ceil(m / ncols)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi=600)
    if nrows == 1 and ncols == 1:
        axes = np.array([[axes]])
    elif nrows == 1:
        axes = np.array([axes])
    elif ncols == 1:
        axes = np.array([[ax] for ax in axes])

    # 展平以便按索引取
    ax_list = axes.ravel()

    for k in range(m):
        ax = ax_list[k]
        i, j = pairs[top_idx[k]]
        fi, fj = feature_names[i], feature_names[j]

        # x: 主特征的原始取值；y: 主特征的 SHAP 值；c: 次特征原始取值
        x_vals = X_sample.iloc[:, i].values
        y_vals = sv_sample.values[:, i]
        c_vals = X_sample.iloc[:, j].values

        sc = ax.scatter(x_vals, y_vals, c=c_vals, s=12, alpha=0.8)
        ax.set_xlabel(f"{fi} (feature value)", fontsize=20)
        ax.set_ylabel(f"SHAP({fi})", fontsize=20)
        ax.set_title(f"{fi} × {fj}  |  |interaction|={scores[top_idx[k]]:.3g}",
                     fontsize=20)
        ax.axhline(0, color="black", linestyle="--", linewidth=0.5)
        ax.tick_params(labelsize=20)
        # 为每个子图单独色条（不同特征范围不同，更直观）
        norm = Normalize(vmin=np.min(c_vals), vmax=np.max(c_vals))
        sm = ScalarMappable(norm=norm)
        sm.set_array([])
        cbar = fig.colorbar(sm, ax=ax)
        cbar.set_label(f"{fj} (feature value)", fontsize=20)

    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

In [ ]:
# ① 绘制 SHAP 特征重要性图（整体）
from scipy.stats import linregress
import seaborn as sns
x_y = [[255, 0.8], [0.00131, 0.11], [1.8, 0.9], [67, 0.6]]
y_ = [-0.4, -0.09, -0.55, -0.6]
def plt_SHAP_la(shap_values, X, i, feature_name, vline):
    if feature_name in X.columns:
        plt.figure(figsize=(6, 6), dpi=1200)
        shap.dependence_plot(
            feature_name,
            shap_values.values,
            X,
            interaction_index=None,  # 或设为其他特征名以观察交互效应
            show=False,
            color="#2c62ad",
        )
        X_reset = X.reset_index(drop=True)
        shap_series = pd.Series(shap_values[feature_name], index=X_reset.index)
        mask = X_reset[feature_name] < vline
        X_below = X_reset[feature_name][mask]
        X_above = X_reset[feature_name][~mask]
        shap_below = shap_series[mask]
        shap_above = shap_series[~mask]
        # 线性回归
        # 对0度之前的数据进行线性回归
        slope_below_0, intercept_below_0, r_value_below_0, p_value_below_0, std_err_below_0 = linregress(X_below, shap_below)
        r_squared_below_0 = r_value_below_0 ** 2
        # 对0度之后的数据进行线性回归
        slope_above_0, intercept_above_0, r_value_above_0, p_value_above_0, std_err_above_0 = linregress(X_above, shap_above)
        r_squared_above_0 = r_value_above_0 ** 2
        # 绘制线性回归线
        plt.text(vline, y_[i], f"{vline}", fontsize=25, color="gray")
        plt.text(x_y[i][0], x_y[i][1], f"Slope:{slope_below_0:.3f}\nR²:{r_squared_below_0:.3f}", fontsize=25, color="#C0635B")
        plt.text(x_y[i][0], x_y[i][1] - x_y[i][1]*0.5, f"Slope:{slope_above_0:.3f}\nR²:{r_squared_above_0:.3f}", fontsize=25, color="#88CDD4")
        # print(f"0度之前的数据线性回归结果：斜率={slope_below_0:.3f}, std={std_err_below_0:.3f}, R²={r_squared_below_0:.3f}")
        # print(f"0度之后的数据线性回归结果：斜率={slope_above_0:.3f}, std={std_err_above_0:.3f}, R²={r_squared_above_0:.3f}")
        sns.regplot(x=X_below, y=shap_below, scatter=False, color="#C0635B", line_kws={"color": "#E6776D", "linewidth": 2})
        sns.regplot(x=X_above, y=shap_above, scatter=False, color="#88CDD4", line_kws={"color": "#88CDD4", "linewidth": 2})
        plt.axhline(0, color="black", linestyle="--", linewidth=1)
        plt.axvline(vline, color="black", linestyle="--", linewidth=1)
        plt.tick_params(labelsize=25)
        plt.xlabel(feature_name, fontsize=30)
        plt.ylabel('SHAP value', fontsize=30)
        plt.tight_layout()
        plt.show()

In [ ]:
# 2. 合并所有气候因子到一个 DataFrame
data = pd.concat([e, t2m, tp, sp, sde, v10, str, strd, ssr, ssrd, ro, rh], axis=1)
scaler = MinMaxScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
# 6. 对每一层的SMTI分别进行建模
layers = smti.columns
shap_values_dict = {}  # 用于存储每层的 SHAP 值
ly_major = ['t2m', 'tp', 'sde', 'rh']
vline = [275.5, 0.00128, 1.7, 72.5]
for i, layer in enumerate(layers):
    print(f"Training model for {layer}")
    # 目标：选择每一层的SMTI
    target = layer
    # 特征：移除目标列
    X = data_scaled
    y = smti[layer]
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    # XGBoost 模型训练
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.01)
    model.fit(X_train, y_train)
    # 预测和评估模型 添加MAE
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = model.score(X_test, y_test)
    rmse = np.sqrt(mse)
    # print(f'R² for {layer}: {r2}')
    # print(f'RMSE for {layer}: {rmse}')
    # print(f'MAE for {layer}: {mae}')
    # 使用 SHAP 解释器分析模型
    explainer = shap.Explainer(model)
    shap_values = explainer(X)
    #
    pd_sp = pd.DataFrame(shap_values.values, columns=X.columns)
    shap_values_dict[layer] = shap_values  # 存储 SHAP 值
    plt_shap(shap_values, X, r2, rmse, mae)
    # plt_SHAP_la(pd_sp, data, i, ly_major[i], vline[i])
    # plot_shap_dependence_grid(shap_values, data, layer,ncols=3, figsize_scale=3.0, color_by=None)
    # plot_shap_interactions(model, X, layer_name=layer)